In [1]:
import torch

# Show details
print(f"PyTorch version: {torch.__version__}, CUDA version: {torch.version.cuda}, GPU available: {torch.cuda.is_available()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PyTorch version: 2.6.0+cu124, CUDA version: 12.4, GPU available: True


In [73]:
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh as mesh
import kaolin.ops.conversions as conversions
import trimesh
import numpy as np
import os
import random
import torch.nn as nn
import torchvision
import open3d as o3d

from datetime import datetime
from itertools import permutations, product
from kaolin.ops.mesh import face_normals
from Normalization import MeshNormalizer
from mesh import Mesh
from pathlib import Path
from render import Renderer
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import color_mesh
import pickle
from scipy.spatial import cKDTree
from utilities.dataset import load_dataset, get_coordinates
from utilities.point_cloud import pointcloud_to_voxel_mesh, project_mesh_labels_to_pointcloud_torch, visualize_affordance_pointcloud

In [10]:
class NeuralHighlighter(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=256, output_dim=2, num_layers=6):
        """
        Args:
            input_dim: usually 3 (x, y, z)
            hidden_dim: size of hidden layers
            output_dim: 2 for [highlight, gray]
            num_layers: total number of linear layers
        """
        super(NeuralHighlighter, self).__init__()

        layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.LayerNorm(hidden_dim)]

        for _ in range(num_layers - 2):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.LayerNorm(hidden_dim))

        layers.append(nn.Linear(hidden_dim, output_dim))
        layers.append(nn.Softmax(dim=1))  # 2-class output

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


def get_clip_model(clipmodel='ViT-L/14', jit=False):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load(clipmodel, device=device, jit=jit)
    print(f"Loaded CLIP model: {clipmodel} on {device} (jit={jit})")
    return model, preprocess


# ================== HELPER FUNCTIONS =============================
def save_final_results(log_dir, name, mesh, mlp, vertices, colors, render, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)
        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)
        sampled_mesh = mesh

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight/255, gray/255)).to(device)
        color_mesh(one_hot, sampled_mesh, colors)
        rendered_images, _, _ = render.render_views(sampled_mesh, num_views=5,
                                                                        show=False,
                                                                        center_azim=0,
                                                                        center_elev=0,
                                                                        std=1,
                                                                        return_views=True,
                                                                        lighting=True,
                                                                        background=background)
        # for mesh
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx==0, highlight, gray)
        mesh.export(os.path.join(log_dir, f"{name}.ply"), extension="ply", color=final_color)
        save_renders(log_dir, 0, rendered_images, name='final_render.jpg')

def clip_loss(rendered_images, text_prompt, clip_transform, clip_model, tokenizer, device, aug_transform=None, n_augs=0):
    """
    """

    loss = 0.0

    # Encode text
    text_tokens = tokenizer([text_prompt]).to(device)
    with torch.no_grad():
        text_features = clip_model.encode_text(text_tokens).float()
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)  # L2 norm

    if n_augs == 0:
        clip_image = clip_transform(rendered_images)
        image_features = clip_model.encode_image(clip_image).float()
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        # Cosine similarity
        loss = -torch.mean(torch.cosine_similarity(image_features, text_features))

    else:
        for _ in range(n_augs):
          aug_image = aug_transform(rendered_images)
          image_encoded = clip_model.encode_image(aug_image)
          loss -= torch.mean(torch.cosine_similarity(image_encoded, text_features))

        loss =  loss / n_augs

    return loss


def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))

In [11]:
# Constrain most sources of randomness
# (some torch backwards functions within CLIP are non-determinstic)

# ==== Set Seed for Determinism ====
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [12]:
def load_vertices(data):
  if type(data) == str:
    mesh = o3d.io.read_triangle_mesh(data)
    mesh.compute_vertex_normals()
    vertices = mesh.sample_points_uniformly(number_of_points=4096)
    return torch.tensor(np.asarray(vertices.points), dtype=torch.float32).to(device)
  else:
    return torch.tensor(data, dtype=torch.float32).to(device)

In [16]:
# ==== Hyperparameters and Settings ====
render_res = 224
learning_rate = 0.00005
n_iter = 2200
n_augs = 1
output_dir = './output/'
clip_model_name = 'ViT-B/16'

In [17]:
# ==== Device ====
render = Renderer(dim=(render_res, render_res))

# ==== CLIP ====
clip_model, preprocess = get_clip_model(clip_model_name)
tokenizer = clip.tokenize

# ==== Normalization and Augmentation ====
clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

clip_transform = transforms.Compose([
    transforms.Resize((render_res, render_res)),
    clip_normalizer
])

augment_transform = transforms.Compose([
    transforms.RandomResizedCrop(render_res, scale=(1, 1)),
    transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
    clip_normalizer
])

# ==== Neural Highlighter ====
mlp = NeuralHighlighter().to(device)
optim = torch.optim.Adam(mlp.parameters(), learning_rate)

# ==== Colors and Other Constants ====
colors = torch.tensor([[204/255, 1., 0.], [180/255, 180/255, 180/255]]).to(device)
background = torch.tensor((1., 1., 1.)).to(device)
n_views = 7
losses = []

Loaded CLIP model: ViT-B/16 on cuda (jit=False)


In [51]:
# Load dataset
dataset = load_dataset("data/full-shape/full_shape_train_data.pkl")

Loaded train_data


In [68]:
affordances = ['grasp', 'wrap grasp', 'pull']

random.shuffle(dataset)
#n_elements = int(0.01 * len(dataset))
n_elements = 1
val_set = dataset[:n_elements]
test_set = dataset[n_elements:(n_elements*2)]

In [53]:
val_counts = {}
test_counts = {}

for item in val_set:
    cls = item["semantic class"]
    if cls not in val_counts:
        val_counts[cls] = 0
    val_counts[cls] += 1

for item in test_set:
    cls = item["semantic class"]
    if cls not in test_counts:
        test_counts[cls] = 0
    test_counts[cls] += 1

sorted_val = sorted(val_counts.items(), key=lambda x: x[1], reverse=True)
sorted_test = sorted(test_counts.items(), key=lambda x: x[1], reverse=True)

print("Validation Set Semantic Class Counts:")
for cls, count in sorted_val:
    print(f"{cls}: {count}")

print("\nTest Set Semantic Class Counts:")
for cls, count in sorted_test:
    print(f"{cls}: {count}")

Validation Set Semantic Class Counts:
Chair: 1

Test Set Semantic Class Counts:
Chair: 1


In [54]:
def build_prompt(semantic_class, affordance):
  temp_prompt = "a gray " + semantic_class + " with highlighted " + affordance + " region"
  return temp_prompt

In [70]:
def optimize_highlighting(sample, affordances, mlp, optimizer, render, clip_model, tokenizer, clip_transform, augment_transform, n_augs, n_iter, colors, background, output_dir, device):
    """
    Optimizes the highlighting process for a given sample and affordances.

    Args:
        sample (dict): The input sample containing mesh and semantic class information.
        affordances (list): List of affordances to highlight.
        mlp (nn.Module): Neural network model for highlighting.
        optimizer (torch.optim.Optimizer): Optimizer for training the model.
        render (Renderer): Renderer for generating views of the mesh.
        clip_model (nn.Module): CLIP model for computing loss.
        tokenizer (function): Tokenizer for text prompts.
        clip_transform (transforms.Compose): Transformations for CLIP input images.
        augment_transform (transforms.Compose): Augmentation transformations for images.
        n_augs (int): Number of augmentations for CLIP loss.
        n_iter (int): Number of optimization iterations.
        colors (torch.Tensor): Tensor of colors for highlighting.
        background (torch.Tensor): Background color tensor.
        output_dir (str): Directory to save results.
        device (torch.device): Device to run computations on.

    Returns:
        torch.Tensor: Predicted class probabilities for vertices.
    """
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    export_path = os.path.join(output_dir, f"run_{timestamp}")
    os.makedirs(os.path.join(export_path, "renders"), exist_ok=True)
    temp_obj_path = "data/outputDemo.obj"

    if "semantic class" not in sample or not isinstance(sample["semantic class"], str):
        raise ValueError(f"Error: Missing or invalid 'semantic class' field in sample: {sample}")

    prompt = build_prompt(sample["semantic class"], affordances[0])  # Use the first affordance for simplicity

    points = get_coordinates(sample, device)
    mesh = pointcloud_to_voxel_mesh(
        points,  # sampled point cloud from Open3D
        resolution=16,
        threshold=0.5,
        export_path=temp_obj_path
    )


    sampled_mesh = Mesh(temp_obj_path)
    MeshNormalizer(sampled_mesh)()
    vertices = sampled_mesh.vertices.clone().detach().to(device).float()

    losses = []
    pred_class = None

    # Optimization loop
    for i in tqdm(range(n_iter)):
        optimizer.zero_grad()
        pred_class = mlp(vertices)  # Predict highlight probabilities
        color_mesh(pred_class, sampled_mesh, colors)  # Color mesh

        # Render and compute loss
        rendered_images, elev, azim = render.render_views(
            sampled_mesh,
            num_views=n_views,
            show=False,
            center_azim=0,
            center_elev=0,
            std=1,
            return_views=True,
            lighting=True,
            background=background
        )
        loss = clip_loss(rendered_images, prompt, clip_transform, clip_model, tokenizer, device, augment_transform, n_augs)
        loss.backward(retain_graph=True)
        optimizer.step()

        with torch.no_grad():
            # Save the loss for logging
            losses.append(loss.item())

        # Log and save intermediate results
        if i % 100 == 0:
            print(f"Last 100 CLIP score: {np.mean(losses[-100:])}")
            save_renders(export_path, i, rendered_images)
            with open(os.path.join(export_path, "training_info.txt"), "a") as f:
                f.write(f"Iter {i}: Prompt: {prompt}, Avg CLIP score: {np.mean(losses[-100:])}, CLIP score: {loss.item()}\n")

    # Final save and cleanup
    save_final_results(export_path, sample["semantic class"], sampled_mesh, mlp, vertices, colors, render, background)
    with open(os.path.join(export_path, "prompt.txt"), "w") as f:
        f.write(prompt)

    if os.path.exists(temp_obj_path):
        os.remove(temp_obj_path)

    return mesh, pred_class

In [76]:
for sample in val_set:
  for affordance in affordances[:1]:
    mesh, pred_class = optimize_highlighting(
        sample=sample,
        affordances=[affordance],
        mlp=mlp,
        optimizer=optim,
        render=render,
        clip_model=clip_model,
        tokenizer=tokenizer,
        clip_transform=clip_transform,
        augment_transform=augment_transform,
        n_augs=n_augs,
        n_iter=n_iter,
        colors=colors,
        background=background,
        output_dir=output_dir,
        device=device
    )

Attribute "face_normals" has not been set and failed to be computed due to: index -1 is out of bounds for dimension 1 with size 0
Unexpected type passed to requires_grad None
Attribute "vertex_normals" has not been set and failed to be computed due to: 'NoneType' object has no attribute 'unsqueeze'
Attribute "face_normals" has not been set and failed to be computed due to: index -1 is out of bounds for dimension 1 with size 0
  0%|          | 2/2200 [00:00<02:56, 12.48it/s]

Last 100 CLIP score: -0.2911253571510315


  5%|▍         | 102/2200 [00:06<02:24, 14.52it/s]

Last 100 CLIP score: -0.26794628441333773


  9%|▉         | 202/2200 [00:13<02:14, 14.84it/s]

...

Last 100 CLIP score: -0.2671652664244175


 41%|████      | 903/2200 [01:05<01:41, 12.83it/s]

Last 100 CLIP score: -0.27071956887841225


 43%|████▎     | 940/2200 [01:07<01:31, 13.83it/s]


KeyboardInterrupt: 

In [71]:
sample = val_set[0]

mesh, pred_class = optimize_highlighting(
    sample=sample,
    affordances=affordances,
    mlp=mlp,
    optimizer=optim,
    render=render,
    clip_model=clip_model,
    tokenizer=tokenizer,
    clip_transform=clip_transform,
    augment_transform=augment_transform,
    n_augs=n_augs,
    n_iter=500,
    colors=colors,
    background=background,
    output_dir=output_dir,
    device=device
)

Attribute "face_normals" has not been set and failed to be computed due to: index -1 is out of bounds for dimension 1 with size 0
Unexpected type passed to requires_grad None
Attribute "vertex_normals" has not been set and failed to be computed due to: 'NoneType' object has no attribute 'unsqueeze'
Attribute "face_normals" has not been set and failed to be computed due to: index -1 is out of bounds for dimension 1 with size 0
  0%|          | 2/500 [00:00<00:39, 12.56it/s]

Last 100 CLIP score: -0.25363409519195557


 21%|██        | 104/500 [00:07<00:26, 14.96it/s]

Last 100 CLIP score: -0.2665786129236221


 41%|████      | 204/500 [00:14<00:19, 15.09it/s]

...

 61%|██████    | 304/500 [00:20<00:13, 15.01it/s]

Last 100 CLIP score: -0.26724553436040877


 80%|████████  | 402/500 [00:27<00:06, 14.84it/s]

Last 100 CLIP score: -0.2671678414940834


100%|██████████| 500/500 [00:33<00:00, 14.78it/s]
